# 🚀 DeepSeek LeetCode Fine-Tuning (Kaggle GPU)

**Settings:** Accelerator → GPU (P100/T4)

This notebook fine-tunes DeepSeek-Coder-6.7B on LeetCode problems using LoRA.

**Key Settings:**
- `dataset_text_field="text"` - Required for SFTTrainer
- `max_seq_length=2048` - Truncate long examples
- `packing=False` - Disable sequence packing

In [ ]:
# Install dependencies
!pip install -q transformers datasets accelerate peft bitsandbytes
!pip install -q trl==0.12.0

In [ ]:
# Verify GPU
import torch
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'NO GPU!'}")
assert torch.cuda.is_available(), "GPU not available! Enable in Settings → Accelerator → GPU"

In [ ]:
# Load Dataset
from datasets import load_dataset, Dataset
ds = load_dataset("LongQ/leetcode_python", split="train")
print(f"✅ Loaded {len(ds)} problems")
print(f"Fields: {ds.column_names}")

In [ ]:
# Format Training Examples
def format_example(ex):
    description = ex.get('problem_description', '')[:2000]
    starter = ex.get('starter_code', '')
    solution = ex.get('solution_code', '')
    
    if not solution or not description:
        return None
    
    text = f"""### Problem:\n{description}\n\n### Starter Code:\n```python\n{starter}\n```\n\n### Solution:\n```python\n{solution}\n```"""
    return {"text": text}

data = [result for ex in ds if (result := format_example(ex)) is not None]
train_ds = Dataset.from_list(data)
print(f"✅ Training examples: {len(train_ds)}")
print(f"Sample:\n{train_ds[0]['text'][:500]}")

In [ ]:
# Load Model with 4-bit Quantization
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

MODEL = "deepseek-ai/deepseek-coder-6.7b-base"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
print(f"✅ Model loaded: {MODEL}")

In [ ]:
# Configure LoRA
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
print(f"✅ LoRA: {trainable:,} trainable / {total:,} total ({100*trainable/total:.2f}%)")

In [ ]:
# Training Configuration
from trl import SFTTrainer, SFTConfig

training_config = SFTConfig(
    output_dir="/kaggle/working/deepseek-leetcode",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    warmup_ratio=0.03,
    weight_decay=0.01,
    logging_steps=25,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,
    fp16=True,
    optim="paged_adamw_8bit",
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    
    # Critical settings
    dataset_text_field="text",
    max_seq_length=2048,
    packing=False,
    
    report_to="none",
    seed=42,
)

print("✅ Training config ready")

In [ ]:
# Train!
trainer = SFTTrainer(
    model=model,
    train_dataset=train_ds,
    args=training_config,
    tokenizer=tokenizer,
)

print("🚀 Starting training...")
trainer.train()
print("✅ Training complete!")

In [ ]:
# Test Generation
test_prompt = """### Problem:
Given an array of integers nums and an integer target, return indices of the two numbers such that they add up to target.

### Starter Code:
```python
class Solution:
    def twoSum(self, nums: List[int], target: int) -> List[int]:
```

### Solution:
```python
"""

inputs = tokenizer(test_prompt, return_tensors="pt").to("cuda")
model.eval()

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        temperature=0.2,
        do_sample=True,
        top_p=0.95,
        pad_token_id=tokenizer.eos_token_id,
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

## 📤 Upload to HuggingFace

Merge LoRA weights into base model and upload as FP16.

In [ ]:
# Login to HuggingFace
!pip install -q huggingface_hub
from huggingface_hub import login
login()

In [ ]:
# Merge LoRA and Upload as FP16
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

MODEL = "deepseek-ai/deepseek-coder-6.7b-base"
ADAPTER = "/kaggle/working/deepseek-leetcode"  # Use latest checkpoint

# Load base model in FP16 on CPU (avoids GPU OOM)
print("Loading base model (FP16)...")
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL, 
    torch_dtype=torch.float16,
    device_map="cpu",
    trust_remote_code=True,
    low_cpu_mem_usage=True
)
tokenizer = AutoTokenizer.from_pretrained(MODEL, trust_remote_code=True)

print("Loading adapter...")
model = PeftModel.from_pretrained(base_model, ADAPTER)

print("Merging...")
model = model.merge_and_unload()

print("Saving locally...")
model.save_pretrained("/kaggle/working/merged", safe_serialization=True)
tokenizer.save_pretrained("/kaggle/working/merged")

print("Uploading to HuggingFace...")
model.push_to_hub("YOUR_USERNAME/deepseek-leetcode-fp16")  # Change this!
tokenizer.push_to_hub("YOUR_USERNAME/deepseek-leetcode-fp16")

print("✅ Done!")

## 🔧 Convert to GGUF (for Ollama)

Run this in a **fresh Kaggle session** to avoid OOM issues.

In [ ]:
# Download from HuggingFace and convert to GGUF
!pip install -q transformers huggingface_hub
!git clone https://github.com/ggerganov/llama.cpp
!pip install -q -r llama.cpp/requirements.txt

from huggingface_hub import snapshot_download
snapshot_download(
    "YOUR_USERNAME/deepseek-leetcode-fp16",  # Change this!
    local_dir="/kaggle/working/merged",
)
print("✅ Downloaded!")

# Convert to GGUF Q8
!python llama.cpp/convert_hf_to_gguf.py /kaggle/working/merged \
    --outfile /kaggle/working/deepseek-leetcode-q8.gguf \
    --outtype q8_0

print("✅ Converted! Download deepseek-leetcode-q8.gguf from Output tab")